In [57]:
import sys

class TwoOptSolver:
    def __init__(self, filename):

        distance_matrix = []

        with open(filename, 'r') as f:
            lines = f.readlines()
            read_distances = False  

            for line in lines: 
                line = line.strip()
                line = line.replace(':', ' ')

                if line.startswith('EOF'):
                    break
                elif line.startswith('EDGE_WEIGHT_SECTION'):
                    read_distances = True
                elif read_distances:
                    elements = line.split()
                    distance_matrix.append([int(e) for e in elements])

        self.graph = distance_matrix
        self.n = len(distance_matrix)
    
    def calc_path_length(self):
        path_cost = 0

        for i in range((self.n - 1)):
            vertex_i = self.best_route[i]
            vertex_j = self.best_route[i + 1]
            cost_ij = self.graph[vertex_i][vertex_j]
            path_cost = path_cost + cost_ij

        # n -> 0
        vertex_i = self.best_route[self.n - 1]
        vertex_j = self.best_route[0]
        cost_ij = self.graph[vertex_i][vertex_j]
        path_cost = path_cost + cost_ij        

        return path_cost
    
    def swap_edges(self, index_i, index_j):
        index_i = index_i + 1

        while index_i < index_j: 
            tmp_vertex = self.best_route[index_i]
            self.best_route[index_i] = self.best_route[index_j]
            self.best_route[index_j] = tmp_vertex
            index_i = index_i + 1
            index_j = index_j - 1


    def two_opt(self):
        
        # route
        self.best_route = list(range(self.n))
        self.best_length = self.calc_path_length()
        improvement_found = True

        while improvement_found == True:
            improvement_found = False

            for i in range((self.n - 1)):

                for j in range((i + 2), self.n):

                    j_1 = (j + 1) % self.n

                    vertex_i = self.best_route[i]
                    vertex_i1 = self.best_route[i + 1]
                    vertex_j = self.best_route[j]
                    vertex_j1 = self.best_route[j_1]


                    r_i_i1 = self.graph[vertex_i][vertex_i1]
                    r_j_j1 = self.graph[vertex_j][vertex_j1]
                    r_i_j = self.graph[vertex_i][vertex_j]
                    r_i1_j1 = self.graph[vertex_i1][vertex_j1]

                    minus = r_i_i1 + r_j_j1
                    plus = r_i_j + r_i1_j1

                    len_delta = plus - minus

                    if len_delta < 0: 
                        # better route found by going i -> j -> ... -> j+1 -> i+1 -> ...
                        improvement_found = True
                        self.best_length = self.best_length + len_delta
                        self.swap_edges(i, j)

        return self.best_length, self.best_route

In [61]:
filename = "../tsplib_converted_instances/03_large/bier127_converted.tsp"

# Solve using 2-Opt 
two_opt_solver = TwoOptSolver(filename)
to_cost, to_path = two_opt_solver.two_opt()

print("\nTwo-Opt-Solver - Minimum Cost:", to_cost)
print("\nPath: ", to_path)


Two-Opt-Solver - Minimum Cost: 136045

Path:  [0, 13, 11, 30, 26, 25, 79, 78, 76, 73, 72, 66, 67, 70, 109, 84, 85, 86, 87, 108, 95, 118, 62, 101, 100, 97, 96, 126, 106, 110, 111, 45, 48, 52, 117, 47, 93, 92, 94, 122, 27, 121, 32, 28, 31, 82, 81, 125, 80, 83, 74, 68, 69, 75, 77, 116, 24, 37, 38, 41, 33, 42, 29, 40, 36, 35, 39, 43, 102, 44, 53, 56, 55, 123, 46, 54, 65, 51, 4, 120, 50, 34, 15, 1, 49, 12, 114, 9, 99, 112, 64, 98, 91, 88, 103, 124, 63, 57, 90, 60, 61, 58, 59, 115, 89, 2, 10, 8, 7, 71, 17, 20, 16, 19, 3, 21, 18, 22, 23, 5, 105, 107, 14, 104, 113, 119, 6]
